# Refer the notebook on Siamese Networks before using this notebook

# Why [Batch Normalisation](https://gist.github.com/shagunsodhani/4441216a298df0fe6ab0)


## Makes the network robust to bad initialization.

## Takes care of Internal Covariate shift.

## Think of it as preprocessing at every layer.

## What is Batch Normalisation

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/shagunsodhani/PyDataConf2017/master/assets/bneq1.png")

## Oops Wrong Equation

In [3]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/shagunsodhani/PyDataConf2017/master/assets/bn_algorithm.PNG")
# Image taken from: http://kratzert.github.io/2016/02/12/understanding-the-gradient-flow-through-the-batch-normalization-layer.html

In [31]:
import pandas as pd
import numpy as np
np.random.seed(1337)

In [32]:
path_to_dataset = "/home/shagun/FortKnox/Quora/quora_duplicate_questions.tsv"
path_to_glove_vectors = "/home/shagun/models/GloVe/glove.6B.100d.txt"

In [33]:
max_len = 50
embedding_dim = 100
# Refer the exploratory notebook to see how max_len value is arrived at

In [34]:
# Load the dataset into a pandas dataframe
df = pd.read_csv(path_to_dataset, delimiter="\t")
print("Total number of question pairs = ", str(len(df)))

Total number of question pairs =  404290


In [35]:
# Let us look at a sample of the dataset
df_sample = df.sample(5)

df_sample

,id,qid1,qid2,question1,question2,is_duplicate
330992,330992,457810,203516,Why do people get white hairs near their temple?,Is there a way to convert some white hair in a...,0
194630,194630,93152,294885,How do you change peoples’ behavior?,What is the best way to change your behavior?,0
102537,102537,169635,169636,What is it like to live in Texas?,What it's like to live in Texas?,1
83320,83320,3767,51558,Can you suggest a best budget phone below 15k?,What is the best phone I can get for below 15k?,1
304304,304304,101579,93681,How can I clear/open/delete messages on Snapchat?,I tried deleting the Snapchat history by selec...,0


## We will play with a very small sample of this dataset to save on time. Feel free to train the network on the entire data later.

In [36]:
df = df.sample(10000)

In [37]:
labels = list(df['is_duplicate'].apply(lambda x: int(x)).values)
labels = np.asarray(labels)

In [38]:
# Create a list of all the question pairs
first_question_list = list(df['question1'].apply(lambda x: str(x)).values)
second_question_list = list(df['question2'].apply(lambda x: str(x)).values)
question_list = list(zip(first_question_list, second_question_list))

In [39]:
print(len(question_list))

10000


In [40]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/shagunsodhani/PyDataConf2017/master/assets/keras-tensorflow-logo.jpg")
# Image taken from: https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

In [41]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from utils.util import *
from keras.layers.wrappers import Bidirectional
from keras.layers import Embedding, Input, GRU, Dense, Activation, Lambda, BatchNormalization
from keras.optimizers import Adam

### Preprocess the data

In [42]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(first_question_list + second_question_list)

In [43]:
sequence1 = pad_sequences(tokenizer.texts_to_sequences(first_question_list), maxlen=max_len)
sequence2 = pad_sequences(tokenizer.texts_to_sequences(second_question_list), maxlen=max_len)

In [44]:
if(len(sequence1) == len(sequence2)):
    print("Good to go")

Good to go


In [45]:
# Basic ML Preprocessing
indices = np.arange(len(sequence1))
np.random.shuffle(indices)
sequence1 = sequence1[indices]
sequence2 = sequence2[indices]
labels = labels[indices]
nb_validation_samples = int(0.3 * len(sequence1))

sequence1_train = sequence1[:-nb_validation_samples]
sequence2_train = sequence2[:-nb_validation_samples]
labels_train = labels[:-nb_validation_samples]
sequence1_val = sequence1[-nb_validation_samples:]
sequence2_val = sequence2[-nb_validation_samples:]
labels_val = labels[-nb_validation_samples:]

In [46]:
print("Number of training examples: " + str(len(sequence1_train)))

Number of training examples: 7000


In [47]:
print("Number of validation examples: " + str(len(sequence1_val)))

Number of validation examples: 3000


### Load embeddings

In [48]:
# Load embeddings
embeddings_index = get_glove_embeddings(path_to_glove_vectors)

### Preparing the embedding matrix which our model would use

In [49]:
# Preparing the embedding matrix which our model would use
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### Encode the question into a vector space using a Bidirectional GRU (or LSTM or whatever RNN you believe in)

In [50]:
def question_encoder():
    return Bidirectional(GRU(units=200), merge_mode='concat', name="bidir_gru")

### The next part is the core of this network and we would walk through it slowly

### Note:  In the implementation, applying this technique usually amounts to insert the BatchNorm layer immediately after fully connected layers, and before non-linearities.

From: http://cs231n.github.io/neural-networks-2/#batchnorm

### Read: https://www.reddit.com/r/MachineLearning/comments/67gonq/d_batch_normalization_before_or_after_relu/

In [51]:
def create_question_network():
    
#     Create an input layer
    sequence_input = Input(shape=(max_len,), dtype='int32', name="input_layer")
    
#     Create an embedding layer
    embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            trainable=False)
    
#     Use the embedding layer we just created
    embedded_sequences = embedding_layer(sequence_input)
    
#     Embeddings are for words, sentences uses encoders
    encoded_question = question_encoder()(embedded_sequences)
    
#     Lets fully connect them
    dense1 = Dense(128)(encoded_question)
    bn1 = BatchNormalization()(dense1)
    relu1 = Activation('relu')(bn1)
    
#    Make it deep
    dense2 = Dense(64)(relu1)
    bn2 = BatchNormalization()(dense2)
    relu2 = Activation('relu')(bn2)    

#     And Deeper
    dense3 = Dense(32)(relu2)
    bn3 = BatchNormalization()(dense3)
    relu3 = Activation('relu')(bn3)

#     Now we are in rythm
    dense4 = Dense(16)(relu3)
    bn4 = BatchNormalization()(dense4)
    relu4 = Activation('relu')(bn4)
    
    output = BatchNormalization()(relu4)
    
    model = Model(inputs=sequence_input, outputs=output)

    return model

### Now we will make the siamese twin

In [52]:
def create_network():
    # network definition
    question_network = create_question_network()
    
#     input to the first head of the network
    input1 = Input(shape=(max_len,))
    
#     input to the second head of the network
    input2 = Input(shape=(max_len,))
    
#     processing the first input
    processed1 = question_network(input1)
    
#     processing the second input
    processed2 = question_network(input2)
    
#     Computing the distance between the transformed inputs.
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed1, processed2])

    
    model = Model(inputs=[input1, input2], outputs=distance)
    return model

In [ ]:
model = create_network()

optimizer = Adam(lr=0.001, clipnorm=5)
model.compile(loss=contrastive_loss, optimizer=optimizer)

for i in range(10):
    model.fit([sequence1_train, sequence2_train], labels_train,
       validation_data=([sequence1_val, sequence2_val], labels_val),
       batch_size=28, epochs=1)
    
    model_labels_train = model.predict([sequence1_train, sequence2_train], batch_size=128)
    print("Training accuracy: "+str(compute_accuracy(model_labels_train, labels_train)))
    
    model_labels_val = model.predict([sequence1_val, sequence2_val], batch_size=128)
    print("Validation accuracy: "+str(compute_accuracy(model_labels_val, labels_val)))

Train on 7000 samples, validate on 3000 samples
Epoch 1/1
7000/7000 [==============================] - 165s - loss: 3.0767 - val_loss: 3.0910
Training accuracy: 0.639714285714
Validation accuracy: 0.626333333333
Train on 7000 samples, validate on 3000 samples
Epoch 1/1
7000/7000 [==============================] - 172s - loss: 1.3316 - val_loss: 1.3903
Training accuracy: 0.667714285714
Validation accuracy: 0.663333333333
Train on 7000 samples, validate on 3000 samples
Epoch 1/1
6048/7000 [========================>.....] - ETA: 21s - loss: 0.9462

In [ ]:
model_labels_val.ravel() < 0.5

In [ ]:
np.mean(np.equal(model_labels_val.ravel() < 0.5, labels_val))

In [ ]:
compute_accuracy()